In [1]:
import pandas as pd
import numpy as np
import subject_data
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta

100%|██████████| 226/226 [00:24<00:00,  9.28it/s]


In [2]:
cgm = subject_data.df_CGM
cgm['time'] = pd.to_datetime(cgm['time'])

def get_corresponding_day(date_time):
        if date_time.hour < 6:  # If before 6 AM, it belongs to the previous day
            return date_time.date() - timedelta(days=1)
        else:
            return date_time.date()

cgm['corresponding_day'] = cgm['time'].apply(get_corresponding_day)

unique_ids = cgm['id'].unique()
# Split the unique IDs into training, validation and test sets 80:10:10
train_ids, test_ids = train_test_split(unique_ids, test_size=0.1, random_state=42)
train_ids, validation_ids = train_test_split(train_ids, test_size=0.111, random_state=42)
# Create the training and test DataFrames based on the split IDs
train_data = cgm[cgm['id'].isin(train_ids)]
validation_data = cgm[cgm['id'].isin(train_ids)]
test_data = cgm[cgm['id'].isin(test_ids)]


In [3]:
def filter_valid_days(group):
    day_values = group[(group['time'].dt.hour >= 6) & (group['time'].dt.hour < 24)]
    filtered_on_day = day_values[day_values.groupby(['corresponding_day']).transform('size') >= 208]
    final_hour_values = filtered_on_day[filtered_on_day['time'].dt.hour >= 23]
    final_hour_values = final_hour_values[final_hour_values['time'].dt.hour < 24]
    filtered_on_final_hour = final_hour_values[final_hour_values.groupby(['corresponding_day']).transform('size') >= 11]
    filtered_days = filtered_on_final_hour['corresponding_day'].unique()
    filtered_group = group[group['corresponding_day'].isin(filtered_days)]
    return filtered_group


In [4]:
cgm = cgm.groupby('id').apply(filter_valid_days)

In [5]:
day = cgm[(cgm['time'].dt.hour >= 6) & (cgm['time'].dt.hour < 24)].rename(columns={'id': 'patient_id'})
night = cgm[(cgm['time'].dt.hour >= 0) & (cgm['time'].dt.hour < 6)].rename(columns={'id': 'patient_id'})

In [6]:
night.reset_index(inplace=True)
night_hypo = night.groupby(['id', 'corresponding_day']).agg({'bgl': lambda x: (x < 70).mean() > 0.04}).rename(columns={'bgl': 'hypo'}).reset_index()

In [91]:
print(len(day['corresponding_day'].unique()))
print(len(night['corresponding_day'].unique()))
print(len(cgm['corresponding_day'].unique()))


501
501
501


In [30]:
# Ensure 'time' is a datetime type and set it as the index
day['time'] = pd.to_datetime(day['time'])
day.set_index('time', inplace=True)

# Resample every 5 minutes and calculate mean, std, and count for 'bgl'
resampled_day = day.groupby(['patient_id', 'corresponding_day']).resample('5T').agg({
    'bgl': ['mean']
}).reset_index()

# Flatten the MultiIndex columns if needed
resampled_day.columns = ['patient_id', 'corresponding_day', 'time', 'mean_bgl']

# Set 'time' back as a column if needed
resampled_day.set_index('time', inplace=True)

# Interpolate 'mean_bgl' for each patient group
for patient_id, group in resampled_day.groupby(['patient_id', 'corresponding_day']):
    print(f'id {patient_id}')
    resampled_day.loc[group.index, 'mean_bgl'] = group['mean_bgl'].interpolate(limit=10, limit_direction='both', method='linear')

id (2, datetime.date(2015, 5, 23))
id (2, datetime.date(2015, 5, 24))
id (2, datetime.date(2015, 5, 25))
id (2, datetime.date(2015, 5, 26))
id (2, datetime.date(2015, 5, 27))
id (2, datetime.date(2015, 5, 28))
id (2, datetime.date(2015, 5, 30))
id (2, datetime.date(2015, 5, 31))
id (2, datetime.date(2015, 6, 1))
id (2, datetime.date(2015, 6, 2))
id (2, datetime.date(2015, 6, 3))
id (2, datetime.date(2015, 6, 4))
id (2, datetime.date(2015, 6, 6))
id (2, datetime.date(2015, 6, 7))
id (2, datetime.date(2015, 6, 8))
id (2, datetime.date(2015, 6, 9))
id (2, datetime.date(2015, 6, 11))
id (2, datetime.date(2015, 6, 12))
id (2, datetime.date(2015, 6, 13))
id (2, datetime.date(2015, 6, 14))
id (2, datetime.date(2015, 6, 15))
id (2, datetime.date(2015, 6, 16))
id (2, datetime.date(2015, 6, 18))
id (2, datetime.date(2015, 6, 19))
id (2, datetime.date(2015, 6, 20))
id (2, datetime.date(2015, 6, 21))
id (2, datetime.date(2015, 6, 22))
id (2, datetime.date(2015, 6, 23))
id (2, datetime.date(2015, 6

In [32]:
resampled_day.head(500)

,patient_id,corresponding_day,mean_bgl
time,,,
2015-05-23 06:00:00,2,2015-05-23,163.0
2015-05-23 06:05:00,2,2015-05-23,159.0
2015-05-23 06:10:00,2,2015-05-23,155.0
2015-05-23 06:15:00,2,2015-05-23,152.0
2015-05-23 06:20:00,2,2015-05-23,147.0
...,...,...,...
2015-05-25 11:15:00,2,2015-05-25,174.0
2015-05-25 11:20:00,2,2015-05-25,165.0
2015-05-25 11:25:00,2,2015-05-25,154.0


In [33]:
resampled_day.to_csv('resampled_day.csv')

In [34]:
night_hypo.to_csv('night_hypo.csv')

In [35]:
resampled_day['mean_bgl'].isna().sum()

0

In [39]:
if 'time' not in resampled_day.columns:
    resampled_day.reset_index(inplace=True)

# Extract only the time part from the datetime column to use as new column headers in the pivot table
resampled_day['time_only'] = resampled_day['time'].dt.time

# Pivot the table using 'time_only' as the columns
pivot_df = resampled_day.pivot_table(
    index=['patient_id', 'corresponding_day'],  # Rows: group by 'patient_id' and 'corresponding_day'
    columns='time_only',                        # Columns: only the time part of the day
    values='mean_bgl',                          # Values: 'mean_bgl' to fill the cells
    aggfunc='first'                             # Aggregation function: 'first' just to avoid duplicates
)

In [40]:
pivot_df.head()

time_only                     06:00:00  06:05:00  06:10:00  06:15:00  \
patient_id corresponding_day                                           
2          2015-05-23            163.0     159.0     155.0     152.0   
           2015-05-24            214.0     215.0     221.0     231.0   
           2015-05-25            115.0     114.0     116.0     118.0   
           2015-05-26            206.0     207.0     207.0     209.0   
           2015-05-27            169.0     171.0     173.0     173.0   

time_only                     06:20:00  06:25:00  06:30:00  06:35:00  \
patient_id corresponding_day                                           
2          2015-05-23            147.0    141.00     134.0     126.0   
           2015-05-24            241.0    252.00     266.0     280.0   
           2015-05-25            121.0    122.00     123.0     126.5   
           2015-05-26            211.0    216.75     222.5     229.0   
           2015-05-27            174.0    182.50     177.0     179.0   

time_only                     06:40:00  06:45:00  ...  23:10:00  23:15:00  \
patient_id corresponding_day                      ...                       
2          2015-05-23            121.0     118.0  ...      79.0      81.0   
           2015-05-24            292.0     304.0  ...     232.0     212.0   
           2015-05-25            121.0     122.0  ...     158.0     154.0   
           2015-05-26            239.0     245.0  ...     146.0     144.0   
           2015-05-27            182.0     178.0  ...     118.0     116.0   

time_only                     23:20:00  23:25:00  23:30:00  23:35:00  \
patient_id corresponding_day                                           
2          2015-05-23             81.0      82.0      83.0      82.0   
           2015-05-24            193.0     173.0     156.0     132.0   
           2015-05-25            148.0     137.0     127.0     121.0   
           2015-05-26            142.0     141.0     140.0     139.0   
           2015-05-27            113.0     112.0     112.0     113.0   

time_only                     23:40:00  23:45:00  23:50:00  23:55:00  
patient_id corresponding_day                                          
2          2015-05-23             84.0      85.0      86.0      81.0  
           2015-05-24            116.0     106.0     100.0      95.0  
           2015-05-25            116.0     107.0      99.0      94.0  
           2015-05-26            138.0     137.0     137.0     137.0  
           2015-05-27            116.0     116.0     115.0     114.0  

[5 rows x 216 columns]

In [41]:
night_hypo.head()

,id,corresponding_day,hypo
0,2,2015-05-23,False
1,2,2015-05-24,False
2,2,2015-05-25,False
3,2,2015-05-26,False
4,2,2015-05-27,False


In [43]:
# Ensure the index of pivot_df is reset to enable merging
pivot_df.reset_index(inplace=True)

# Rename 'patient_id' to 'id' in pivot_df to match the column name in night_hypo for consistency
pivot_df.rename(columns={'patient_id': 'id'}, inplace=True)

# Merge pivot_df with night_hypo
# Assuming night_hypo has columns 'id' and 'corresponding_day' that match with pivot_df
merged_df = pd.merge(pivot_df, night_hypo, on=['id', 'corresponding_day'], how='left')

# Check the merged DataFrame
merged_df.head()

,index,id,corresponding_day,06:00:00,06:05:00,06:10:00,06:15:00,06:20:00,06:25:00,06:30:00,...,23:15:00,23:20:00,23:25:00,23:30:00,23:35:00,23:40:00,23:45:00,23:50:00,23:55:00,hypo
0,0,2,2015-05-23,163.0,159.0,155.0,152.0,147.0,141.00,134.0,...,81.0,81.0,82.0,83.0,82.0,84.0,85.0,86.0,81.0,False
1,1,2,2015-05-24,214.0,215.0,221.0,231.0,241.0,252.00,266.0,...,212.0,193.0,173.0,156.0,132.0,116.0,106.0,100.0,95.0,False
2,2,2,2015-05-25,115.0,114.0,116.0,118.0,121.0,122.00,123.0,...,154.0,148.0,137.0,127.0,121.0,116.0,107.0,99.0,94.0,False
3,3,2,2015-05-26,206.0,207.0,207.0,209.0,211.0,216.75,222.5,...,144.0,142.0,141.0,140.0,139.0,138.0,137.0,137.0,137.0,False
4,4,2,2015-05-27,169.0,171.0,173.0,173.0,174.0,182.50,177.0,...,116.0,113.0,112.0,112.0,113.0,116.0,116.0,115.0,114.0,False


In [44]:
# Drop the 'index' column
merged_df.drop(columns=['index'], inplace=True)

# Set 'id' and 'corresponding_day' as the index
merged_df.set_index(['id', 'corresponding_day'], inplace=True)

# Display the updated DataFrame to check the new indexing
merged_df.head()


06:00:00  06:05:00  06:10:00  06:15:00  06:20:00  \
id corresponding_day                                                     
2  2015-05-23            163.0     159.0     155.0     152.0     147.0   
   2015-05-24            214.0     215.0     221.0     231.0     241.0   
   2015-05-25            115.0     114.0     116.0     118.0     121.0   
   2015-05-26            206.0     207.0     207.0     209.0     211.0   
   2015-05-27            169.0     171.0     173.0     173.0     174.0   

                      06:25:00  06:30:00  06:35:00  06:40:00  06:45:00  ...  \
id corresponding_day                                                    ...   
2  2015-05-23           141.00     134.0     126.0     121.0     118.0  ...   
   2015-05-24           252.00     266.0     280.0     292.0     304.0  ...   
   2015-05-25           122.00     123.0     126.5     121.0     122.0  ...   
   2015-05-26           216.75     222.5     229.0     239.0     245.0  ...   
   2015-05-27           182.50     177.0     179.0     182.0     178.0  ...   

                      23:15:00  23:20:00  23:25:00  23:30:00  23:35:00  \
id corresponding_day                                                     
2  2015-05-23             81.0      81.0      82.0      83.0      82.0   
   2015-05-24            212.0     193.0     173.0     156.0     132.0   
   2015-05-25            154.0     148.0     137.0     127.0     121.0   
   2015-05-26            144.0     142.0     141.0     140.0     139.0   
   2015-05-27            116.0     113.0     112.0     112.0     113.0   

                      23:40:00  23:45:00  23:50:00  23:55:00   hypo  
id corresponding_day                                                 
2  2015-05-23             84.0      85.0      86.0      81.0  False  
   2015-05-24            116.0     106.0     100.0      95.0  False  
   2015-05-25            116.0     107.0      99.0      94.0  False  
   2015-05-26            138.0     137.0     137.0     137.0  False  
   2015-05-27            116.0     116.0     115.0     114.0  False  

[5 rows x 217 columns]

In [45]:
merged_df.to_csv('CGM_RESAMPLED.csv')

In [48]:
merged_df = pd.merge(pivot_df, night_hypo, on=['id', 'corresponding_day'], how='left')


In [57]:
unique_ids = cgm['id'].unique()
# Split the unique IDs into training, validation and test sets 80:10:10
train_ids, test_ids = train_test_split(unique_ids, test_size=0.1, random_state=42)
train_ids, validation_ids = train_test_split(train_ids, test_size=0.111, random_state=42)


train_data = merged_df[merged_df['id'].isin(train_ids)]
validation_data = merged_df[merged_df['id'].isin(validation_ids)]
test_data = merged_df[merged_df['id'].isin(test_ids)]

In [62]:
train_data.to_csv('train_data.csv')
validation_data.to_csv('validation_data.csv')
test_data.to_csv('test_data.csv')